## Match raw CF file to final location set and elementary flow

In [1]:
import os
import json
import pandas as pd
import brightway2 as bw
from IPython.display import display
import pickle
import numpy as np

# Land Use
### Load data

In [29]:
path_to_file = os.path.abspath('./raw_data/PSLglobal_modified_region_name.xlsx')
%time excel_file = pd.ExcelFile(path_to_file,engine='openpyxl')

CPU times: user 153 ms, sys: 12.4 ms, total: 165 ms
Wall time: 163 ms


In [30]:
excel_file.sheet_names

['Explanations',
 'Occupation_Ecoregion',
 'Occupation_Country',
 'Occupation_Continent',
 'Transformation_Ecoregion',
 'Transformation_Country',
 'Transformation_Continent',
 'Taxaspecific_Occupation',
 'TaxasspecificTransformation']

In [31]:
sheets = [sheet for sheet in excel_file.sheet_names if 
          any((n in sheet for n in ('Country', 'Continent', 'Ecoregion')))]

combine the same type of CF with different region resolution

In [32]:
def parse_land_use_sheets(sheetname, excel_file):
    skip = 2 if 'Ecoregion' in sheetname else 1
    df = pd.read_excel(excel_file, sheet_name=sheetname, skiprows=skip, index_col=0, header=[0,1])
    df_median = df.xs('Median', axis=1, level=1, drop_level=True)
    return df_median

In [33]:
cf_dict = {}
categories = ('Transformation', 'Occupation')
for category in categories:
    dfs = {sheet:parse_land_use_sheets(sheet, excel_file) for sheet in sheets if category in sheet}
    print(category)
    for name, df in dfs.items():
        print(name, df.shape)
    df = pd.concat(dfs.values(), axis=0)
    df = df[~df.index.duplicated()] # remove duplicates from index
    df['Natural'] = 0
    cf_dict[category] = df
    print(df.shape, '\n')

Transformation
Transformation_Ecoregion (805, 6)
Transformation_Country (244, 6)
Transformation_Continent (8, 6)
(1055, 7) 

Occupation
Occupation_Ecoregion (805, 6)
Occupation_Country (244, 6)
Occupation_Continent (8, 6)
(1055, 7) 



In [34]:
for category, df in cf_dict.items():
    df.to_csv(f'intermediate_data/{category.lower()}_median.csv')

### Match to ecoinvent

#### match regions: final_loc -> co_code -> co_name -> row in CF table

In [35]:
with open('/home/sipeng2/2021RA_ESD/RegEcoData/data/tech_vector/final_loc_set.p', 'rb') as i:
    final_loc_set = pickle.load(i)

In [36]:
eco_geo = pd.read_excel (r'../../../ecoinvent_database/eiv3.7_geographies_names_coordinates_shortcuts_overlaps.xlsx',
                         sheet_name='list of geographies',keep_default_na=False, engine='openpyxl')
shortcut_eco_loc = eco_geo.set_index('shortname')['name'].to_dict()

In [45]:
for category in categories:
    
    CF_final_loc_df = pd.DataFrame(columns=cf_dict[category].columns.tolist())
    df = cf_dict[category]
    
    #add missing CF: Kosovo -> Europe. Hong Kong, Taiwan, Province of China-> China
    df.loc['Kosovo'] = df.loc['Europe'] 
    df.loc['Hong Kong'] = df.loc['Asia'] 
    df.loc['Taiwan, Province of China'] = df.loc['Asia']  
    
    # final_loc -> co_code -> co_name -> CF
    for loc in final_loc_set:
        if loc=='Canada without Quebec':
            co_code = 'CA'
        else:
            co_code = loc[:2]
        co = shortcut_eco_loc[co_code]
        CF_final_loc_df.loc[loc]=df.loc[co]
    CF_final_loc_df.loc['World average']=df.loc['World average']
    
    CF_final_loc_df = CF_final_loc_df.sort_index()
    CF_final_loc_df.to_csv(f'intermediate_data/landuse_{category.lower()}_CF_with_final_location.csv')

#### match impact categories: bio flow code -> bio flow name -> column in CF table

In [38]:
def load_json(path):
    with open(path, 'r') as i:
        json_dict = json.load(i)
    return json_dict

In [39]:
lu_category_to_elemflows = load_json('matching_dictionaries/landuse_category_to_ei_elementary_flows.json')
lu_category_to_elemflows

{'Occupation, annual crop': ['Occupation', 'Annual crops'],
 'Occupation, annual crop, flooded crop': ['Occupation', 'Annual crops'],
 'Occupation, annual crop, greenhouse': ['Occupation', 'Annual crops'],
 'Occupation, annual crop, irrigated': ['Occupation', 'Annual crops'],
 'Occupation, annual crop, irrigated, extensive': ['Occupation',
  'Annual crops'],
 'Occupation, annual crop, irrigated, intensive': ['Occupation',
  'Annual crops'],
 'Occupation, annual crop, non-irrigated': ['Occupation', 'Annual crops'],
 'Occupation, annual crop, non-irrigated, extensive': ['Occupation',
  'Annual crops'],
 'Occupation, annual crop, non-irrigated, intensive': ['Occupation',
  'Annual crops'],
 'Occupation, arable land, unspecified use': ['Occupation', 'Annual crops'],
 'Occupation, arable, conservation tillage (obsolete)': ['Occupation',
  'Annual crops'],
 'Occupation, arable, conventional tillage (obsolete)': ['Occupation',
  'Annual crops'],
 'Occupation, arable, reduced tillage (obsolete

In [59]:
len(lu_category_to_elemflows)

180

In [40]:
bw.projects.set_current('regeco')
bio = bw.Database('biosphere3').load()

In [41]:
# all bio flow with occupation or transformation in the name
lu_flow_eco = []
for k, v in bio.items():
    if 'Occupation' in v['name'] or 'Transformation' in v['name']:
        lu_flow_eco.append(v['name'])
set(lu_flow_eco).difference(set(lu_category_to_elemflows.keys())) # the following two are not in dict

{'Transformation, from unknown', 'Transformation, to unknown'}

In [42]:
# the missing two are also not used as exchanges in our study
with open('../../data/final_matrix/biosphere_index.p', 'rb') as i:
    bio_index = pickle.load(i)
act_flows = []
for ind in bio_index:
    bio_key = ('biosphere3',ind)
    if 'Occupation' in bio[bio_key]['name'] or 'Transformation' in bio[bio_key]['name']:
        act_flows.append(bio[bio_key]['name'])
set(lu_flow_eco).difference(act_flows)

{'Occupation, annual crop, flooded crop',
 'Occupation, annual crop, irrigated, extensive',
 'Occupation, arable, conservation tillage (obsolete)',
 'Occupation, arable, conventional tillage (obsolete)',
 'Occupation, arable, reduced tillage (obsolete)',
 'Occupation, bare area (non-use)',
 'Occupation, cropland fallow (non-use)',
 'Occupation, field margin/hedgerow',
 'Occupation, forest, primary (non-use)',
 'Occupation, forest, secondary (non-use)',
 'Occupation, forest, unspecified',
 'Occupation, heterogeneous, agricultural',
 'Occupation, lake, natural (non-use)',
 'Occupation, permanent crop, irrigated, extensive',
 'Occupation, permanent crop, non-irrigated, extensive',
 'Occupation, river, natural (non-use)',
 'Occupation, seabed, natural (non-use)',
 'Occupation, snow and ice (non-use)',
 'Occupation, wetland, coastal (non-use)',
 'Occupation, wetland, inland (non-use)',
 'Transformation, from annual crop, flooded crop',
 'Transformation, from annual crop, irrigated, extensiv

In [46]:
code_to_name = {b['code']:b['name'] for b in bio.values() if b['name'] in lu_category_to_elemflows}
to_from = {code:-1 if name.split(' ')[1]=='from' else 1 for code,name in code_to_name.items()}

In [48]:
for category in categories:
    CF_final_loc_df = pd.read_csv(f'intermediate_data/landuse_{category.lower()}_CF_with_final_location.csv', index_col = 0, keep_default_na=False)
    CF_final_loc_biocode_df = pd.DataFrame(index = CF_final_loc_df.index.tolist())
    for code,name in code_to_name.items(): 
        if category in name:
            CF_final_loc_biocode_df[code] = CF_final_loc_df[tuple(lu_category_to_elemflows[name])[1]]*to_from[code]
    CF_final_loc_biocode_df = CF_final_loc_biocode_df.reindex(sorted(CF_final_loc_biocode_df.columns), axis=1)
    CF_final_loc_biocode_df.to_csv(f'prepared_characterization_factors/prepared_land_use_{category.lower()}_CF.csv')

# Water use 
## Load data

In [51]:
path_to_file = os.path.abspath('./raw_data/AWARE_country_regions_world_april2016_modified_region_name.xlsx')
%time excel_file = pd.ExcelFile(path_to_file,engine='openpyxl')

CPU times: user 29.9 ms, sys: 6.73 ms, total: 36.7 ms
Wall time: 35.3 ms


In [52]:
excel_file.sheet_names

['AWARE', 'AWARE monthly', 'Spatio-Temporal Variability']

In [53]:
aware = pd.read_excel(excel_file, sheet_name='AWARE', skiprows=2, usecols='A:D')
aware.dropna(axis=0, how='all', inplace=True) #drop three rows where all values are na (blank)
aware.fillna(axis=1, method='ffill', inplace=True) #propagate non-null values forward, column-wise 
aware_unknown = aware[['COUNTRY', 'UNKNOWN']]
aware_unknown = aware_unknown[aware_unknown.applymap(pd.np.isreal)['UNKNOWN']] # remove those with text
aware_unknown.set_index('COUNTRY', inplace=True)
aware_cf = aware_unknown['UNKNOWN']
len(aware_cf)

/home/sipeng2/miniconda3/envs/regeco/lib/python3.6/site-packages/ipykernel_launcher.py:5: FutureWarning: The pandas.np module is deprecated and will be removed from pandas in a future version. Import numpy directly instead
  """


227

## Match to ecoinvent

### regions

In [55]:
aware_final_loc_df = pd.DataFrame(columns=aware_unknown.columns.tolist())
df = aware_unknown

#add missing CF: South Sudan -> RAF. Kosovo -> Europe. Curaçao -> RLA. Hongkong, Taiwan -> RAS
df.loc['Kosovo'] = df.loc['RER'] 
df.loc['South Sudan'] = df.loc['RAF'] 
df.loc['Curaçao'] = df.loc['RLA'] 
df.loc['Hong Kong'] = df.loc['RAS'] 
df.loc['Taiwan, Province of China'] = df.loc['RAS']  

# final_loc -> co_code -> co_name -> CF
for loc in final_loc_set:
    if loc=='Canada without Quebec':
        co_code = 'CA'
    else:
        co_code = loc[:2]
    co = shortcut_eco_loc[co_code]
    aware_final_loc_df.loc[loc]=df.loc[co]
aware_final_loc_df.loc['World average']=df.loc['World, UNKNOWN']

aware_final_loc_df = aware_final_loc_df.sort_index() #by index (final loc)
aware_final_loc_df.to_csv(f'intermediate_data/aware_CF_with_final_location.csv')

### Elementary flows

In [23]:
aware_flow_factors = load_json('matching_dictionaries/wateruse_elementary_flows_factors.json')
aware_flow_factors = {tuple(k.split('|')):v for k,v in aware_flow_factors.items()}

In [24]:
aware_flow_factors

{('Fresh water (obsolete)', "('water', 'surface water')"): -1.0,
 ('Water, cooling, unspecified natural origin',
  "('natural resource', 'in water')"): 1.0,
 ('Water, lake', "('natural resource', 'in water')"): 1.0,
 ('Water, river', "('natural resource', 'in water')"): 1.0,
 ('Water, turbine use, unspecified natural origin',
  "('natural resource', 'in water')"): 1.0,
 ('Water, unspecified natural origin',
  "('natural resource', 'in water')"): 1.0,
 ('Water, well, in ground', "('natural resource', 'in water')"): 1.0,
 ('Water', "('water', 'ground-')"): -1.0,
 ('Water', "('water', 'ground-, long-term')"): -1.0,
 ('Water', "('water', 'surface water')"): -1.0,
 ('Water', "('water',)"): -1.0}

In [25]:
# check water relevant elementary flows
water_flow_eco = []
for k, v in bio.items():
    if 'Water' in v['name'] and 'water' in v['categories']: #only consider water compartment
#     if 'Water' in v['name'] and 'in water' in v['categories']:
        water_flow_eco.append((v['name'], v['categories'],v['type']))
water_flow_eco.sort()
water_flow_eco #only consider fresh water
# set(lu_flow_eco).difference(set(lu_category_to_elemflows.keys())) # the following two are not in dict

[('Water', ('water',), 'emission'),
 ('Water', ('water', 'fossil well'), 'emission'),
 ('Water', ('water', 'ground-'), 'emission'),
 ('Water', ('water', 'ground-, long-term'), 'emission'),
 ('Water', ('water', 'ocean'), 'emission'),
 ('Water', ('water', 'surface water'), 'emission')]

In [56]:
code_to_name = {b['code']:(b['name'], str(b['categories'])) for b in bio.values() if 
                (b['name'], str(b['categories'])) in aware_flow_factors}
# aware_final_loc_df

In [57]:
aware_CF_final_loc_biocode_df = pd.DataFrame(index = aware_final_loc_df.index.tolist())
for code,name in code_to_name.items():
    aware_CF_final_loc_biocode_df[code] = aware_final_loc_df['UNKNOWN']*aware_flow_factors[name]
cols = aware_CF_final_loc_biocode_df.columns.tolist()    
cols.sort()
aware_CF_final_loc_biocode_df = aware_CF_final_loc_biocode_df[cols]
aware_CF_final_loc_biocode_df.to_csv(f'prepared_characterization_factors/prepared_aware_CF.csv')